### ***SIMULACION***  ⚡


In [5]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

x_1 = 70 #numero de unidades del producto 1
x_2 = 70 #numero de unidades del producto 2
T_simulacion = 5*24*30
#T_simulacion = 5*30*24 #tiempo transcurrido en la simulacion en horas
Tp = 7*24 #cada cuanto pide (cada semana)
lista = {'tc': 0,  # tiempo en el que ha llegado un cliente
         'tpc': 0, # tiempo en el que se ha comprado un pedido
         'tp': 0}  # tiempo en el que ha llegado un pedido
R = 0 #beneficio esperado
P1 = 1000 #numero de unidades max del producto 1
P2 = 1500 #numero de unidades max del producto 2
Nc = 0 #numero de clientes satisfechos
Nnc = 0 # numero de clientes no satisfechos
t0 = 0 # numero de tiempo que el intervalo esta a cero
C = 0 #coste total por pedidos
H = 0 #coste total por almacenamiento
lambda_poisson = 0.5
demanda = [1, 2, 3, 4] #posibles demandas del producto
probab_1 = [0.3, 0.4, 0.2, 0.1] #probabilidades de demanda del producto 1
probab_2 = [0.2, 0.2, 0.4, 0.2] #probabilidades de demanda del producto 2
r_1 = 2.5 #coste al publico del producto 1
r_2 = 3.5 #coste al publico del producto 2
h = 0.0002 #precio sumado por producto y unidad de tiempo
mu = 48 #tiempo de media que tarda un pedido
sigma = 3.5 #desviacion tipica de lo que tarda un pedido
K = 100 #coste fijo del proveedor
n_descuent_1 = 600 #unidades mayores a estas obtienen descuento en producto 1
p1_1 = 1 # precio si son menos de 600 uds del producto 1
p1_2 = 0.75 # precio si son mas de 600 uds del producto 1
n_descuent_2 = 800
p2_1 = 1.5
p2_2 = 1.25
L = 0 #lo que tarda en llegar el pedido desde que se compra
Lref = 48 #tiempo media de llegada del pedido
lim_penal = 3 #a partir de estas horas de retraso del pedido, penalizacion
penal = 0.0003 # la penalizacion en el precio si llega tarde/pronto 
t_real = 0
var_aux = 0 # instante en el que el almacen se vacia completamnete
ts = 0 #tiempo de simulacion

#Vectores para la representación gráfica de los niveles de inventario de los dos tipos de producto a lo largo del tiempo

tiempos_1 = [0]
niveles_1 = [70]
tiempos_2 = [0]
niveles_2 = [70]

# datos_grafica [producto 1 o 2] [tiempo (0) o nivel(1)] [i]  
datos_grafica = ["",
                 [[0],[70]], # tiempo y nivel producto 1
                 [[0],[70]]  # tiempo y nivel producto 2
                ]

#----------------------------------------------------------------------
#             Metodos
#----------------------------------------------------------------------

def rutina_llegada_cliente(ts):
  global H, h, t_real, x_1, x_2, Nc, Nnc, var_aux, R, Y, y_1, y_2
  global r_1, r_2, var_aux, T_simulacion, tiempos_1, tiempos_2
  global niveles_1, niveles_2
  
  #Aumenta el coste de almacenamiento
  H += (ts-t_real)*h*(x_1+x_2)
  t_real = ts
  
  #Generamos demanda del cliente
  demanda_1 = np.random.choice(demanda, 1, p=probab_1)[0]
  demanda_2 = np.random.choice(demanda, 1, p=probab_2)[0]
  
  
  #Si hay suficiente almacenado, esta satisfecho
  if demanda_1<=x_1 and demanda_2<=x_2 :
    R += demanda_1*r_1 + demanda_2*r_2 #sube el beneficio
    x_1 -= demanda_1   #baja el inventario
    x_2 -= demanda_2
    Nc += 1 #cliente satisfecho
 
  #Si no hay suficiente almacenado de algun producto, no esta satisfecho
  else:
    Nnc += 1 #cliente no satisfecho
    if(demanda_1<=x_1):
      R += demanda_1*r_1 
      x_1 -= demanda_1
    elif(demanda_2<=x_2):
      R += demanda_2*r_2 
      x_2 -= demanda_2
    
  # Si se ha vaciado del todo (y antes no estaba vacio) guardamos el tiempo actual
  if x_2 == 0 and x_1 == 0 and var_aux == 0 :
    var_aux = t_real
    
  #Generamos el tiempo que tarda en llegar el siguiente cliente
  Y = stats.poisson.rvs(lambda_poisson, size=1)[0] 
  
  # si el cliente llega antes de acabar la simulacion, se simula
  if Y+t_real < T_simulacion:
    lista['tc'] = t_real+Y 

def rutina_llegada_pedido(ts):
  global H, K, h, t_real, C, t0, var_aux, x_1, x_2, y_1, y_2
  global p1_1, p1_2, p2_1, p2_1, penal, var_aux
  global tiempos_1, tiempos_2, niveles_1, niveles_2
  
  #Aumenta el coste de almacenamiento
  H += (ts-t_real)*h*(x_1+x_2)
  t_real = ts
  
  #Aumenta el nivel de inventario
  x_1 += y_1
  x_2 += y_2
  
  #Si son muchas unidades, descuento en el precio
  Ci_1 = K + y_1 * p1_1 if y_1<=n_descuent_1 else K + y_1 * p1_2
  Ci_2 = K + y_2 * p2_1 if y_2<=n_descuent_2 else K + y_2 * p2_2
  
  #Si llega pronto es mas caro, si llega tarde, es mas barato
  if L < Lref-lim_penal:
    C += (Ci_1 + Ci_2) * (1 + penal)
  elif L > Lref+lim_penal:
    C += (Ci_1 + Ci_2) * (1 - penal)
  else:
    C += Ci_1 + Ci_2
  
  #Ya no quedan productos por llegar
  y_1 = 0
  y_2 = 0
  
  # Si estaba vacio el invenario (se vacio en el instante var_aux),
  # aumenta el tiempo que ha estado vacio.
  if var_aux > 0:
    t0 += t_real - var_aux
    var_aux = 0

def rutina_compra_pedido(ts):
  global H, x_1, x_2, t_real, y_1, y_2, h, t_real
  global P1, P2, lista, T_simulacion

  # Aumenta el coste de almacenamiento
  H += (ts-t_real)*h*(x_1+x_2)
  t_real = ts
  
  #Cantidad a pedir es lo que falta para llenar el almacen
  y_1 = P1 - x_1
  y_2 = P2 - x_2

  #Generamos cuanto va a tardar en llegar el pedido
  L = np.random.normal(mu, sigma, 1)[0]

  # actualizamos el tiempo de llegada del pedido y el tiempo de siguiente compra
  if L+t_real < T_simulacion:
    lista['tp'] = t_real + L
    
  if t_real+Tp < T_simulacion:
    lista['tpc'] = t_real + Tp

def simul_main(periodicidad,nivel_1,nivel_2):
  global x_1, x_2, T_simulacion , Tp, lista, R, P1, P2, Nc, Nnc, t0, C, H
  global lambda_poisson, demanda, probab_1, probab_2, r_1, r_2, h, mu, sigma
  global K, n_descuent_1, p1_1, p1_2, p2_1, p2_2, n_descuent_2, L, Lref
  global lim_penal, penal, treal, var_aux, ts
  
  #Reseteamos las variables

  x_1 = 70 #numero de unidades del producto 1
  x_2 = 70 #numero de unidades del producto 2
  T_simulacion = 5*24*30
  #T_simulacion = 5*30*24 #tiempo transcurrido en la simulacion en horas
  Tp = periodicidad #cada cuanto pide (cada semana)
  lista = {'tc': 0,  # tiempo en el que ha llegado un cliente
          'tpc': 0, # tiempo en el que se ha comprado un pedido
          'tp': 0}  # tiempo en el que ha llegado un pedido
  R = 0 #beneficio esperado
  P1 = nivel_1 #numero de unidades max del producto 1
  P2 = nivel_2 #numero de unidades max del producto 2
  Nc = 0 #numero de clientes satisfechos
  Nnc = 0 # numero de clientes no satisfechos
  t0 = 0 # numero de tiempo que el intervalo esta a cero
  C = 0 #coste total por pedidos
  H = 0 #coste total por almacenamiento
  lambda_poisson = 0.5
  demanda = [1, 2, 3, 4] #posibles demandas del producto
  probab_1 = [0.3, 0.4, 0.2, 0.1] #probabilidades de demanda del producto 1
  probab_2 = [0.2, 0.2, 0.4, 0.2] #probabilidades de demanda del producto 2
  r_1 = 2.5 #coste al publico del producto 1
  r_2 = 3.5 #coste al publico del producto 2
  h = 0.0002 #precio sumado por producto y unidad de tiempo
  mu = 48 #tiempo de media que tarda un pedido
  sigma = 3.5 #desviacion tipica de lo que tarda un pedido
  K = 100 #coste fijo del proveedor
  n_descuent_1 = 600 #unidades mayores a estas obtienen descuento en producto 1
  p1_1 = 1 # precio si son menos de 600 uds del producto 1
  p1_2 = 0.75 # precio si son mas de 600 uds del producto 1
  n_descuent_2 = 800
  p2_1 = 1.5
  p2_2 = 1.25
  L = 0 #lo que tarda en llegar el pedido desde que se compra
  Lref = 48 #tiempo media de llegada del pedido
  lim_penal = 3 #a partir de estas horas de retraso del pedido, penalizacion
  penal = 0.0003 # la penalizacion en el precio si llega tarde/pronto 
  t_real = 0
  var_aux = 0 # instante en el que el almacen se vacia completamnete
  ts = 0 #tiempo de simulacion
  
  # Iniciamos siumlacion
  ts = 0
  lista['tc'] = 4000
  lista['tp'] = 4000
  lista['tpc'] = Tp


  #Generamos el tiempo que tarda en llegar el primer cliente
  Z = stats.poisson.rvs(lambda_poisson, size=1)[0]

  #Si el tiempo si pasa del limite T, la simulación se acaba
  if(Z > T_simulacion): return -1

  #print('** Llega el primer cliente')
  rutina_llegada_cliente(Z)
  
  #Repetir si siguen llegando clientes o siguen llegando pedidos
  while (lista['tc']!=4000 or lista['tp']!=4000) and ts <= T_simulacion:
    
    #Si el siguiente evento es la llegada de un cliente
    if lista['tc'] <= lista['tpc'] and lista['tc'] <= lista['tp']:
      ts = lista['tc']
      lista['tc'] = 4000
      rutina_llegada_cliente(ts)

    #Si el siguiente evento es la compra de un pedido 
    elif(lista['tpc']<=lista['tc'] and lista['tpc']<=lista['tp']):
      ts = lista['tpc']
      lista['tpc'] = 4000
      rutina_compra_pedido(ts)

    #Si el siguiente evento es una llegada de pedido
    elif(lista['tp']<=lista['tc'] and lista['tp']<=lista['tpc']):
      ts = lista['tp']
      lista['tp'] = 4000
      rutina_llegada_pedido(ts)
      
  benef = R - C - H                #Beneficios
  cl_satisf = Nc / (Nc + Nnc) *100 #Porcentaje de clientes satisfechos
  t0_tot = t0 / T_simulacion       #Tiempo que ha estado el almacen vacio
  
  return benef

### ***RECOCIDO SIMULADO***  🎯

In [11]:
import random
import math

def recocido_simulado(initial_state):
    """Peforms simulated annealing to find a solution"""
    L = 10 #Num de iteraciones donde mantenemos la temperatura
    initial_temp = 10000
    #final_temp = .1
    alpha = 0.95 #Ritmo de decrecimiento de la temperatura
    iteraciones = 1 #Contador de iteraciones
    max_iteraciones = 100

    # Primera iteracion
    current_temp = initial_temp
    current_state = initial_state
    current_state["benef"] = get_benef(current_state)
    best_state = current_state
    
    while iteraciones <= max_iteraciones:
        # Check if neighbor is best so far
        neighbor = get_neighbor(current_state)
        neighbor["benef"] = get_benef(neighbor)

        benef_diff = abs(neighbor["benef"] - current_state["benef"])
        prob = math.exp(-benef_diff / current_temp)

        # if the new solution is better, accept it
        # if the new solution is not better, accept it with a probability of e^(-cost/temp)
        if benef_diff > 0 or random.uniform(0, 1) < prob:
            current_state = neighbor
                
        print("\tIt."+str(iteraciones)+
              "\tTemp.: "+str(int(current_temp))+
              "\tProb.: "+str(prob)+
              "\tBeneficio: "+str(current_state["benef"])+
              "\tParametros: "+str(current_state["periodicidad"])+", "+
                               str(current_state["nivel_1"])+", "+
                               str(current_state["nivel_2"]))
        
        # decrement the temperature
        if 1000 > iteraciones > L:
          current_temp = current_temp * alpha
        
        if best_state["benef"] < current_state["benef"]:
          best_state = current_state

        iteraciones+=1

    return best_state

#triggers simulation and returns benefits
def get_benef(state):
    return simul_main(state["periodicidad"], state["nivel_1"], state["nivel_2"])

def get_neighbor(state):

    """Returns neighbors of the argument state for your solution."""
    new_periodicidad = state["periodicidad"] +  np.random.normal(0,2,1)[0]
    new_nivel_1 = state["nivel_1"] +  round(np.random.normal(0,2,1)[0])
    new_nivel_2 = state["nivel_2"] +  round(np.random.normal(0,2,1)[0])

    neighbor = {'periodicidad':new_periodicidad ,  
                      'nivel_1': new_nivel_1,
                      'nivel_2': new_nivel_2 }

    return neighbor

In [12]:
initial_state = {'periodicidad':168 ,  
                      'nivel_1': 1500,
                      'nivel_2': 1000 }

recocido_simulado(initial_state)

	It.1	Temp.: 10000	Prob.: 0.9406999569483021	Beneficio: 53808.2824770433	Parametros: 168.04108186733444, 1502, 1001
	It.2	Temp.: 10000	Prob.: 0.8848982810206023	Beneficio: 55031.108249841505	Parametros: 170.6603082068873, 1498, 996
	It.3	Temp.: 10000	Prob.: 0.9589365308130152	Beneficio: 55450.4121394975	Parametros: 172.36424475860673, 1499, 992
	It.4	Temp.: 10000	Prob.: 0.8592916303798072	Beneficio: 53933.942991949094	Parametros: 173.4409842022751, 1495, 994
	It.5	Temp.: 10000	Prob.: 0.9386994310903134	Beneficio: 53301.34353494294	Parametros: 173.4850882362494, 1496, 993
	It.6	Temp.: 10000	Prob.: 0.960359055362359	Beneficio: 53705.82401938642	Parametros: 171.2650993881172, 1497, 997
	It.7	Temp.: 10000	Prob.: 0.7943358821943688	Beneficio: 56008.312836899626	Parametros: 169.2286105982208, 1497, 996
	It.8	Temp.: 10000	Prob.: 0.9602231738948476	Beneficio: 55602.417349592244	Parametros: 169.98681433586424, 1496, 995
	It.9	Temp.: 10000	Prob.: 0.9745694125100192	Beneficio: 55344.822012336306	

{'periodicidad': 158.81473547858315,
 'nivel_1': 1494,
 'nivel_2': 996,
 'benef': 56580.04041800314}

0.7788007830714049